In [1]:
import pandas as pd

In [2]:
species_name = "Danio reiro"
rna_type = "mRNA"
rna_type_for_fasta = "protein_coding"

In [3]:
df = pd.read_csv('Download_data_RP/Download_data_RP.csv', on_bad_lines = 'skip', sep = '\t' )

/tmp/ipykernel_5938/1951409903.py:1: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Download_data_RP/Download_data_RP.csv', on_bad_lines = 'skip', sep = '\t' )


In [4]:
filt = (df['Category1'] == rna_type) & (df['Species1'] == species_name) 

In [5]:
df_filtered = df.loc[filt]
df_filtered.loc

In [6]:
    df2 = pd.unique(df_filtered['Interactor1.Symbol'].values.ravel())
    print("Get unique values from multiple columns:\n",df2)

Get unique values from multiple columns:
 []


In [7]:
from Bio import SeqIO


In [65]:
def type_assertion(record):
    parts = record.description.split('|') 
    if(len(parts) == 5 and parts[4] == rna_type_for_fasta):
        return "true"
    else:
        return "false"

In [66]:
def get_accession1(record):
    parts = record.description.split('|') 
    assert len(parts) == 5 and parts[4] == rna_type_for_fasta
    return parts[3], parts[1]

In [67]:
fasta_dict = {}
for record in SeqIO.parse("Fastas/mart_export.txt", "fasta"):
    if(type_assertion(record) == "true"):
        gene, code = get_accession1(record)
        fasta_dict.update({gene : code})

In [68]:
def get_accession2(record):
    parts = record.description.split('|') 
    assert len(parts) == 5 and parts[4] == rna_type_for_fasta
    return parts[1]

In [69]:
record_dict = SeqIO.to_dict(SeqIO.parse("Fastas/mart_export.txt", "fasta"), key_function = get_accession2)
len(record_dict)

299

In [70]:
output_list = []
print(record_dict[fasta_dict[df2[0]]].seq)


GGTCCTATAGTGTAGTGGTTATCACTTTCGGTTCTGATCCGAACAACCCCAGTTCGAATCCGGGTGGGACCT


In [71]:
handle = open("Fastas/mart_export.txt")
for x in range(0, len(df2), 1):
    if df2[x] not in fasta_dict.keys():
        continue
    output_list.append([df2[x], record_dict[fasta_dict[df2[x]]].seq])
print(output_list)
handle.close()

[['CDC65', Seq('GGTCCTATAGTGTAGTGGTTATCACTTTCGGTTCTGATCCGAACAACCCCAGTT...CCT')], ['EMT1', Seq('GCTTCAGTAGCTCAGTAGGAAGAGCGTCAGTCTCATAATCTGAAGGTCGAGAGT...GCA')], ['SUF1', Seq('GGGCGGTTAGTGTAGTGGTTATCATCCCACCCTTCCAAGGTGGGGACACGGGTT...TCA')], ['SUF11', Seq('GGGCGTGTGGTCTAGTGGTATGATTCTCGCTTTGGGTGCGAGAGGCCCTGGGTT...CCC')], ['SUF2', Seq('GGGCGTGTGGTCTAGAGGTATGATTCTCGCTTAGGGTGCGGGAGGTCCCGGGTT...CCC')], ['SUF5', Seq('GCGCAAGTGGTTCAGTGGTTAGAATTTATGCTTCCCAAGCATGAGGCCCGGGTT...GCA')], ['SUF7', Seq('GGGCGTGTGGTCTAGTGGTATGATTCTCGCTTTGGGTGCGAGAGGCCCTGGGTT...CCC')], ['SUF8', Seq('GGGCGTGTGGTCTAGTGGTATGATTCTCGCTTTGGGTGCGAGAGGCCCTGGGTT...CCC')], ['SUF9', Seq('GGGCGTGTGGTCTAGTGGTATGATTCTCGCTTTGGGTGCGAGAGGCCCTGGGTT...CCC')], ['SUP19', Seq('GGCACTATGGCCGAGTGGTTAAGGCGACAGACTTGAAATCTGTTGGGCTCTGCC...TCG')], ['SUP2', Seq('CTCTCGGTAGCCAAGTTGGTTTAAGGCGCAAGACTGTAAATCTTGAGATCGGGC...AGA')], ['SUP56', Seq('GGTTGTTTGGCCGAGCGGTCTAAGGCGCCTGATTCAAGCTCAGGTATCGTAAGA...CCA')], ['TGA1', Seq('GGGCACATGGCGCAGTTGGTAGCGCGCTTCCCT

In [72]:
# Create a DataFrame from the list
out_data_frame = pd.DataFrame(output_list, columns=['Gene Name', 'Sequence'])
# Save DataFrame to CSV file
out_data_frame.to_csv("outputs/" + species_name + "_" + rna_type + '.csv', index=False, encoding='utf-8')